In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import yaml
import sys

In [ ]:
# Paths to necessary files: requirements.txt, config.yaml, and the document (PDF) to be processed
req_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/requirements.txt'

config_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/config.yaml'

doc_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/privacy-pmi.pdf'

# Load and read the requirements.txt file and config.yaml file
with open(req_path, 'r') as file:
   requirements = file.read().splitlines()

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# Install the required Python packages listed in requirements.txt
for package in requirements:
    !pip install {package}

In [ ]:
from llama_index.core import load_index_from_storage

In [ ]:
import gradio as gr
sys.path.append('/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/src')
from ragpipelines import *
config

In [ ]:
# Extract necessary configurations such as API key, document path, and model names from the YAML file
api_key=config['openai_api_key']
document_path=doc_path
llm_model=config['llm_model']
embed_model=config['embed_model']

In [ ]:
# Initialize the pipelines
basic_rag_pipeline = BasicRAGPipeline(
    api_key,
    document_path,
    llm_model,
    embed_model)

sentence_window_pipeline = SentenceWindowRAGPipeline(
    api_key,
    document_path,
    llm_model,
    embed_model)

auto_merging_pipeline = AutoMergingRAGPipeline(
    api_key,
    document_path,
    llm_model,
    embed_model)

auto_merging_pipeline_with_prompting = AutoMergingRAGPipelineWithPrompting(
    api_key,
    document_path,
    llm_model,
    embed_model
)

In [ ]:
# Define a function to query different RAG pipelines based on the user's selection
def query_pipeline(query, history, selected_pipeline):
    if selected_pipeline == "Basic RAG":
        response = basic_rag_pipeline.query(query)
    elif selected_pipeline == "Sentence Window (Rerank)":
        sentence_window_pipeline.rerank_enabled = True
        response = sentence_window_pipeline.query(query)
    elif selected_pipeline == "Sentence Window (No Rerank)":
        sentence_window_pipeline.rerank_enabled = False
        response = sentence_window_pipeline.query(query)
    elif selected_pipeline == "Auto Merging":
        response = auto_merging_pipeline.query(query)
    elif selected_pipeline == "Auto Merging with Prompting":
        response = auto_merging_pipeline_with_prompting.query(query)
    history = history or []
    history.append((query, response))
    return history, history, ""

In [ ]:
# Gradio interface 1: 5 pipelines displayed at once for comparison
def create_demo():
    # Initialize history storage for each pipeline
    histories = {
        "Basic RAG": [],
        "Sentence Window (Rerank)": [],
        "Sentence Window (No Rerank)": [],
        "Auto Merging": [],
        "Auto Merging with Prompting": []  # Add this line
    }

    def query_pipeline_interface(query, pipeline_selector):
        # Retrieve the correct history for the selected pipeline
        history = histories[pipeline_selector]

        # Use the provided query_pipeline function to get the response
        history, _, _ = query_pipeline(query, history, pipeline_selector)

        # Ensure the response is a string before appending it to the history
        history[-1] = (query, str(history[-1][1]))

        # Store the updated history back into the histories dictionary
        histories[pipeline_selector] = history

        # Return the updated history for the selected pipeline
        return history, ""


    with gr.Blocks(title="PrivacyBot - Display 1", theme="Soft") as demo:
        gr.Markdown("<h1 style='text-align: center;'>🔒🤖 PrivacyBot 🤖🔒</h1>")
        gr.Markdown("<p style='text-align: center;'>Go ahead and ask me any questions you have about how PMI treats your personal data.</p>")

        # Pipeline selector at the top
        with gr.Row():
            pipeline_selector = gr.Dropdown(
                choices=list(histories.keys()),
                value="Basic RAG",
                label="Select RAG Pipeline"
            )

        with gr.Row():
            with gr.Column(scale=1):
                chat_history = gr.Chatbot(elem_id='chatbot', height=500)

        with gr.Row():
            with gr.Column(scale=0.8):
                text_input = gr.Textbox(
                    show_label=False,
                    placeholder="Ask anything about how PMI treats your personal data.",
                    container=False
                )
            with gr.Column(scale=0.2):
                submit_button = gr.Button('Send')

        # Update the chat history and process the input
        submit_button.click(query_pipeline_interface, inputs=[text_input, pipeline_selector], outputs=[chat_history, text_input])
        text_input.submit(query_pipeline_interface, inputs=[text_input, pipeline_selector], outputs=[chat_history, text_input])

        # Update chat history when a new pipeline is selected
        pipeline_selector.change(lambda pipeline: histories[pipeline], inputs=[pipeline_selector], outputs=[chat_history])

    return demo

In [ ]:
# Gradio interface 2: you select 1 pipeline at once, history is saved when you move from 1 pipeline to the other
def create_demo():
    with gr.Blocks(title="PrivacyBot - Display 2", theme="Soft") as demo:
        gr.Markdown("<h1 style='text-align: center;'>🔒🤖 PrivacyBot 🤖🔒</h1>")
        gr.Markdown("<p style='text-align: center;'>Go ahead and ask me any questions you have about how PMI treats your personal data.</p>")

        # First row with three columns
        with gr.Row():
            with gr.Column():
                basic_rag_history = gr.Chatbot(elem_id='chatbot_basic_rag', label="Basic RAG Response", height=600)
                basic_rag_input = gr.Textbox(show_label=False, placeholder="Ask anything about PMI's privacy policy.", container=False)
                basic_rag_button = gr.Button('Submit')

            with gr.Column():
                sentence_window_rerank_history = gr.Chatbot(elem_id='chatbot_sentence_window_rerank', label="Sentence Window (Rerank) Response", height=600)
                sentence_window_rerank_input = gr.Textbox(show_label=False, placeholder="Ask anything about PMI's privacy policy.", container=False)
                sentence_window_rerank_button = gr.Button('Submit')

            with gr.Column():
                sentence_window_no_rerank_history = gr.Chatbot(elem_id='chatbot_sentence_window_no_rerank', label="Sentence Window (No Rerank) Response", height=600)
                sentence_window_no_rerank_input = gr.Textbox(show_label=False, placeholder="Ask anything about PMI's privacy policy.", container=False)
                sentence_window_no_rerank_button = gr.Button('Submit')

        # Second row with two columns
        with gr.Row():
            with gr.Column():
                auto_merging_history = gr.Chatbot(elem_id='chatbot_auto_merging', label="Auto Merging Response", height=600)
                auto_merging_input = gr.Textbox(show_label=False, placeholder="Ask anything about PMI's privacy policy.", container=False)
                auto_merging_button = gr.Button('Submit')

            with gr.Column():
                auto_merging_prompting_history = gr.Chatbot(elem_id='chatbot_auto_merging_prompting', label="Auto Merging with Prompting Response", height=600)
                auto_merging_prompting_input = gr.Textbox(show_label=False, placeholder="Ask anything about PMI's privacy policy.", container=False)
                auto_merging_prompting_button = gr.Button('Submit')

        # Function to handle submission
        def submit_handler(input_text, history, state_name):
            if input_text.strip():  # Ensure input is not empty
                return query_pipeline(input_text, history, state_name)
            else:
                return history, history, ""

        # Set up the pipeline-specific query and output connections
        basic_rag_button.click(submit_handler, inputs=[basic_rag_input, basic_rag_history, gr.State("Basic RAG")], outputs=[basic_rag_history, basic_rag_history, basic_rag_input])
        sentence_window_rerank_button.click(submit_handler, inputs=[sentence_window_rerank_input, sentence_window_rerank_history, gr.State("Sentence Window (Rerank)")], outputs=[sentence_window_rerank_history, sentence_window_rerank_history, sentence_window_rerank_input])
        sentence_window_no_rerank_button.click(submit_handler, inputs=[sentence_window_no_rerank_input, sentence_window_no_rerank_history, gr.State("Sentence Window (No Rerank)")], outputs=[sentence_window_no_rerank_history, sentence_window_no_rerank_history, sentence_window_no_rerank_input])
        auto_merging_button.click(submit_handler, inputs=[auto_merging_input, auto_merging_history, gr.State("Auto Merging")], outputs=[auto_merging_history, auto_merging_history, auto_merging_input])
        auto_merging_prompting_button.click(submit_handler, inputs=[auto_merging_prompting_input, auto_merging_prompting_history, gr.State("Auto Merging with Prompting")], outputs=[auto_merging_prompting_history, auto_merging_prompting_history, auto_merging_prompting_input])

        # Connect the Textbox submission (Enter key) to the Button click event
        basic_rag_input.submit(submit_handler, inputs=[basic_rag_input, basic_rag_history, gr.State("Basic RAG")], outputs=[basic_rag_history, basic_rag_history, basic_rag_input])
        sentence_window_rerank_input.submit(submit_handler, inputs=[sentence_window_rerank_input, sentence_window_rerank_history, gr.State("Sentence Window (Rerank)")], outputs=[sentence_window_rerank_history, sentence_window_rerank_history, sentence_window_rerank_input])
        sentence_window_no_rerank_input.submit(submit_handler, inputs=[sentence_window_no_rerank_input, sentence_window_no_rerank_history, gr.State("Sentence Window (No Rerank)")], outputs=[sentence_window_no_rerank_history, sentence_window_no_rerank_history, sentence_window_no_rerank_input])
        auto_merging_input.submit(submit_handler, inputs=[auto_merging_input, auto_merging_history, gr.State("Auto Merging")], outputs=[auto_merging_history, auto_merging_history, auto_merging_input])
        auto_merging_prompting_input.submit(submit_handler, inputs=[auto_merging_prompting_input, auto_merging_prompting_history, gr.State("Auto Merging with Prompting")], outputs=[auto_merging_prompting_history, auto_merging_prompting_history, auto_merging_prompting_input])

    return demo

In [ ]:
def launch_interface():
    demo = create_demo()
    demo.launch(share=True)

In [ ]:
launch_interface()